# STA663 Final Project Report

### Abstract

We implement the Bayesian hierarchical clustering algorithm described in the paper by Heller and Ghahramani (2005) and optimized the algorithm's speed using Cython. 

### Background

Hierarchical clustering is an unsupervised learning algorithm that organizes data into a binary tree. By cutting the tree at various heights, one can acquire clustering structures of the data. The traditional hierarchical clustering algorithm is the  agglomerative hierarchical clustering method. This algorithm starts with each data point being in it's own cluster and uses a bottom-up approach of iteratively merging the most similar clusters until all the data has been grouped into one cluster. The similarity of clusters is evaluated using a pre-specified distance measure (e.g. Euclidean distance).

The major drawback of the traditional hierarchical clustering method is that it does not provide intuitive guidance to choosing the correct number of clusters or the appropriate distance measure. Furthermore, the algorithm does not define a probabilistic model for the data, thus it is not able to give predictions or cluster new data points into existing clusters.

Heller and Ghahramani (2005) proposed the Bayesian hierarchical clustering model to overcome those aforementioned problems. The model is similar to the traditional hierarchical clustering method, but Bayesian hierarchical clustering merges data using the marginal likelihood of data. The major advantage of Bayesian hierarchical clustering over the traditional method is that it has a natural way to choose the correct number of clusters, and it can also give predictions of the probability of assigning new data into existing clusters. On the other hand, the disadvantage of the algorithm is that it is more computationally intensive since it requires calculating the marginal likelihood at each step. Bayesian hierarchical clustering works best when the user has no prior knowledge on the number of clusters or when the user wants to predict the probability of new data belonging to any existing clusters.

### Description of algorithm

The Bayesian hierarchical clustering algorithm starts with every data point being in its own node. Afterwards, it iteratively merge the nodes with the highest merge probability until there is only a single node left (all data points in the same node). Merge probability is defined as the the ratio of the marginal likelihood of the two nodes being in the same cluster against the marginal likelihood of the nodes being partitioned in all possible ways which does not violate the existing tree structure. The tree can then be cut at the step where the merge probability is less than 50%.

In mathematical terms, if we represent the hypothesis that the nodes are in the same cluster by $H_1^k$ and the tree structure using $T$, then we can write our the algorithm:

1.Input: data = {$x^{(1)}, ..., x^{(n)}$}, model(likelihood) $p(x \mid \theta)$, prior $p(\theta \mid \beta)$

2.Initialize: number of clusters $c = n$, $D_i = $ {$x^{(i)}$} for $i = 1, ..., n$

3.while $c > 1$ do

Find the pair of clusters $D_i$ and $D_j$ that has the highest merge probability: $r_k = \frac{\pi_k p(D_k \mid H_1^k)}{p(D_k \mid T_k)}$ 
        
Merge $D_i, D_j$ into $D_k$, $T_i$, and $T_j$ into $T_k$

Delete $D_i, D_j$ and $c = c - 1$

end while

To calculate the merge probabilities, we need to compute $p(D_k \mid T_k)$, which is
$$
p(D_k \mid T_k) = \pi_k p(D_k \mid H_1^k) + (1 - \pi_k) p(D_i \mid T_i)p(D_j \mid T_j)
$$
The term $\pi_k$ can be calculated by

1.Initialize: each leaf $i$ with $d_i = \alpha, \pi_i = 1$ 

2.for each internal node $k$ do

$d_k = \alpha \Gamma(n_k) + d_{left_k}d_{right_k}$

$\pi_k = \frac{\alpha \Gamma(n_k)}{d_k}$

end for

### Optimization for performance

In [42]:
%load_ext cython

In [43]:
%%cython
import numpy as np
import itertools as it
from scipy.special import gamma
from scipy.special import multigammaln
from numpy.math cimport INFINITY
from scipy.stats import multivariate_normal
cimport numpy as np


def BHC_cython(data, likelihood, alpha):
    """`
    Bayesian hierarchical clustering algorithm, direct implementation from paper.
    
    Parameters
    ----------
    data : 2D numpy array
        Data with rows as observations and columns as variables.
    likelihood : function
        Function that returns the marginal likelihood of data in BHC model.
    alpha : float
        Concentration parameter in model.
        
    Returns
    -------
    T_record : list(list(int))
        Cluster structure in each iteration.
    rk_record : list(float)
        Merge probability (rk) in each iteration.
    """
    
    cdef int n, c, i, j
    cdef float max_rk
    cdef int merge_index1, merge_index2
    cdef double pi_k, rk
    cdef int node_index, item_index
    cdef double likelihood_i, likelihood_j, likelihood_k
    cdef T_record, rk_record
    
    n = data.shape[0]
    p = data.shape[1]
    c = n
    D = dict((index, Node(obs.reshape(-1, p), alpha)) for index, obs in enumerate(data))
    T = list(range(n))
    T_record = list(T)
    rk_record = [1]
    
    while c > 1:
        max_rk = -INFINITY
        node_merge = None
        left_merge = -1
        right_merge = -1
        
        for i, j in it.combinations(D.keys(), 2):
            Di = D[i]
            Dj = D[j]
            Dk = Node.merge_node(Di, Dj)
            
            likelihood_i = likelihood(Di.data)
            likelihood_j = likelihood(Dj.data)
            likelihood_k = likelihood(Dk.data)
            
            pi_k = Dk.pi_k

            rk = (pi_k * likelihood_k) / (pi_k * likelihood_k + (1 - pi_k) * likelihood_i * likelihood_j)

            if rk > max_rk:
                max_rk = rk
                node_merge = Dk
                left_merge = i
                right_merge = j
        
        #delete merged nodes and store new node
        del D[right_merge]
        D[left_merge] = node_merge

        #store the current tree structure and value of rk
        for item_index, node_index in enumerate(T):
            if node_index == right_merge:
                T[item_index] = left_merge
        T_record.append(list(T))
        rk_record.append(max_rk)
        
        c -= 1
        
    return T_record, rk_record
        

class Node(object):
    """
    Node class used in Bayesian hierarchical clustering algorithm. Main purpose is to store values of dk and pi_k for each node.
    
    Attributes
    ----------
    data : 2D numpy array
        Data with rows as observations and columns as variables.
    dk : float
        Some kind of number for computing probabilities
    pi_k : float
        For to compute merge probability
        
    Methods
    -------
    __init__(self, data, likelihood, alpha = 1)
        Instantiation operation.
    merge_node(cls, node1, node2, alpha = 1)
        Method that merges two Nodes into one new Node and return the new Node.
    """
    
    def __init__(self, data, alpha = 1, dk = 1, pi_k = 1):
        """
        Instantiation operation.
        
        Parameters
        ----------
        data : 2D numpy array
            Data with rows as observations and columns as variables.
        likelihood : function
            Function that returns the likelihood of data, sampling distribution in BHC model.
        alpha : float
            Concentration parameter in model.
        log_dk : float
            Cached probability variable. Do not define if the node is a leaf.
        log_pi : float
            Cached probability variable. Do not define if the node is a leaf.
        """
        
        #initialized according to paper
        self.data = data
        self.dk = dk
        self.pi_k = pi_k

    @classmethod
    def merge_node(cls, node1, node2, alpha = 1):
        """
        Merge two Nodes into one new Node and return the new Node.
        
        Parameters
        ----------
        node1 : Node
            First Node.
        node2 : Node
            Second Node.
        """

        cdef np.ndarray[dtype = double, ndim = 2] data
        cdef double nk
        
        data = np.vstack((node1.data, node2.data))

        nk = data.shape[0]
        dk = alpha * gamma(nk) + node1.dk * node2.dk
        pi_k = alpha * gamma(nk) / dk

        return cls(data, alpha, dk, pi_k)

TypeError: '>=' not supported between instances of 'NoneType' and 'str'

### Applications to simulated data sets

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [48]:
n = 10
y1 = np.random.multivariate_normal([0, 0], np.eye(2) * 0.1, n)
y2 = np.random.multivariate_normal([1, 0], np.eye(2) * 0.3, n)
y3 = np.random.multivariate_normal([0, 1], np.eye(2) * 0.5, n)
y4 = np.random.multivariate_normal([-1, 1], np.eye(2) * 0.7, n)
y5 = np.random.multivariate_normal([0, -1], np.eye(2) * 0.9, n)
simu_true_cluster = np.repeat(list(range(1, 6)), 10)

simulate_data = np.vstack((y1, y2, y3, y4, y5))

In [45]:
def purity_score(cluster_true, cluster_pred):
    """
    Function that returns the purity scores of each cluster given the predicted clusters and the true clusters
    """
    
    def most_common(lst):
        """Helper function that finds the most frequent item in a list"""
        return max(set(lst), key = lst.count)
    
    purity = []
    for cluster in set(cluster_pred):
        np.where(np.array(cluster_pred) == cluster)
        cluster_element = [cluster_true[index] for index in np.where(np.array(cluster_pred) == cluster)[0]]
        dominant_item = most_common(cluster_element)
        purity.append(cluster_element.count(dominant_item) / len(cluster_element))
        
    return purity

### Applications to real datasets
We tested our BHC algorithm on the glass dataset mentioned in the original paper, as well as the famous iris dataset by Fisher.

In [4]:
glass = pd.read_csv("data/glass.data", index_col=0, header=None)
iris = pd.read_csv("data/iris.data", header=None)
glass_x = glass.iloc[:, 0:9]
iris_x = iris.iloc[:, 0:4]

### Comparative analysis with competing algorithms
In this section, we will apply the traditional hierarchical clustering with single linkage and K-means clustering to our simulated and real world datasets and compare their performance with the Bayesian hierarchical clustering algorithm we implemented.

In [46]:
purity_df = pd.DataFrame(columns = ("Hierarchical clustering", "K-means", "BHC"))
from scipy.cluster.hierarchy import linkage, cut_tree
HC = linkage(simulate_data, 'single')
HC_cluster = cut_tree(HC, 5)
purity_HC_simu = np.mean(purity_score(simu_true_cluster, HC_cluster.flatten()))

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 5, random_state=0).fit(simulate_data)
kmeans_cluster = kmeans.labels_
purity_KM_simu = np.mean(purity_score(simu_true_cluster, kmeans_cluster))



### Conclusion

### References

Heller, K. A., & Ghahramani, Z. (2005). Bayesian Hierarchical Clustering. Neuroscience, 6(section 2), 297–304. doi:10.1145/1102351.1102389

Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.